In [ ]:
## General Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sn
import importlib
from moviepy import ImageSequenceClip

In [138]:
## System Imports
import os
import sys
from pathlib import Path

In [ ]:
## Changing Directory to Root
'''
print(os.getcwd())

parent = os.path.dirname(os.getcwd())
os.chdir(parent)

print(os.getcwd())
'''

c:\Users\Zach\Projects\Current\2025 NFL Big Data Bowl\notebooks
c:\Users\Zach\Projects\Current\2025 NFL Big Data Bowl


In [141]:
## Modules
from src.modules import util, classes

In [142]:
## Module Reload
importlib.reload(util)
importlib.reload(classes)

<module 'src.modules.classes' from 'c:\\Users\\Zach\\Projects\\Current\\2025 NFL Big Data Bowl\\src\\modules\\classes.py'>

#### Data

In [4]:
## Load each dataset
games = pd.read_csv("data/raw/games.csv")
player_play = pd.read_csv("data/raw/player_play.csv")
players = pd.read_csv("data/raw/players.csv")
plays = pd.read_csv("data/raw/plays.csv")

## Load weekly data
week1 = pd.read_csv("data/raw/tracking_week_1.csv")
week2 = pd.read_csv("data/raw/tracking_week_2.csv")
week3 = pd.read_csv("data/raw/tracking_week_3.csv")
week4 = pd.read_csv("data/raw/tracking_week_4.csv")

In [42]:
## Observing Data
util.print_to_text('games',games.head().to_string())
util.print_to_text('player_play',player_play.head().to_string())
util.print_to_text('players',players.head().to_string())
util.print_to_text('plays',plays.head().to_string())
util.print_to_text('week1',week1.head(10).to_string())

In [113]:
## Bears week 1 game -- QB plays -- first play
gameId = int(games[(games['homeTeamAbbr']=='CHI') & (games['week']==1)].reset_index().loc[:,'gameId'][0])

bears_passing_plays = player_play[(player_play['gameId']==id) & (player_play['teamAbbr']=='CHI') & (player_play['hadDropback']==1)].sort_values(by='playId').reset_index().drop('index',axis=1)
first_play = bears_passing_plays.loc[0,:]
playId = int(first_play['playId'])

home_abr = str(games[(games['homeTeamAbbr']=='CHI') & (games['week']==1)].reset_index().loc[:,'homeTeamAbbr'][0])
visitor_abr = str(games[(games['homeTeamAbbr']=='CHI') & (games['week']==1)].reset_index().loc[:,'visitorTeamAbbr'][0])

## play by play, for first play
pbp = week1[(week1['gameId']==id) & (week1['playId']==playId)]
pbp_qb = pbp[pbp['nflId']==int(first_play['nflId'])]

util.print_to_text('pbp_qb',pbp_qb.to_string())

#### Visualizing Pocket

In [143]:
pbp_coordinate_data = pbp[['gameId','playId','nflId','displayName','club','frameId','x','y']]

In [145]:
play = classes.Play(gameId=gameId,playId=playId,home_abr=home_abr,visitor_abr=visitor_abr)
frame_count = np.max(pbp_coordinate_data['frameId'])

for f in range(1,frame_count+1):
    pbp_frame = pbp_coordinate_data[pbp_coordinate_data['frameId']==f]
    frame = classes.Frame(gameId=gameId,playId=playId,frameId=f,home_abr=home_abr,visitor_abr=visitor_abr)
    
    for row in pbp_frame.itertuples(index=False):
        frame.add_moment(name=row.displayName,nflId=row.nflId,club=row.club,frameId=row.frameId,x=row.x,y=row.y)
    
    play.add_frame(frame)
    
for frame in play.frames:
    frame.to_png()

In [146]:
image_folder = "debug/frames1/"
image_files = [f"{image_folder}/{gameId}_{playId}_{str(i)}_field.png" for i in range(1,frame_count+1)]

clip = ImageSequenceClip(image_files, fps=5)
clip.write_videofile("debug/frames1.mp4", codec="libx264")

MoviePy - Building video debug/frames1.mp4.
MoviePy - Writing video debug/frames1.mp4



MoviePy - Done !
MoviePy - video ready debug/frames1.mp4


In [147]:
os.startfile(Path.cwd() / "debug" / "frames1.mp4")